# TASK - XÁC ĐỊNH HIỆU XE Ô TÔ TỪ ẢNH

Công ty Nguyên Phong muốn phát triển một hệ thống phân loại xe ô tô từ ảnh để tổng hợp thị trường xe hơi của các hãng như Honda, Suzuki, Toyota, Mitsubishi, Mazda, Huyndai, KIA, Peugeot, Volvo, Volkswagen, LandRover, Nissan, Mercedes, VinFast
Thành phần cốt lõi của hệ thống phân loại này là module
- Input: ảnh chụp xe ô tô
- Output: nhãn thuộc một trong các loại: Honda, Suzuki, Toyota, Mitsubishi, Mazda, Huyndai, KIA, Peugeot, Volvo, Volkswagen, LandRover, Nissan, Mercedes, VinFast, không có ô tô, ô tô khác




## Ví dụ:
- Input: <IMG SRC='https://i1-vnexpress.vnecdn.net/2023/05/10/Vios-2023-10_1683690128.jpg?w=2400&h=0&q=100&dpr=1&fit=crop&s=LPsTjgivZ2T5xKJYQiVjHg&t=image'>


- Output: Toyota

## Dữ liệu
Mỗi SV/nhóm SV thu thập dữ liệu theo yêu cầu sau:
- Đặt tên thư mục theo cấu trúc
    + Honda (các ảnh của xe ô tô hiệu Honda)
    + Mazda
    + Mitsubishi
    + Suzuki
    + Toyota
    + Hyundai
    + KIA
    + VinFast
    + Others (các ảnh của xe ô tô loại khác)
- Mỗi SV cần thu thập trung bình 10 ảnh khác nhau/loại, thu thập từ Internet (Google Images Search, Bing Images Search, tự chụp thực tế)
- Tên ảnh đặt theo quy ước: <DS MSSV cách nhau bằng ->.<Tên hiệu xe>.<Số TT ảnh>
    + 20221234-20222345.Honda.1
- Độ đa dạng (variations)
    + Góc nhìn: đầu xe hướng về trái, về phải, về trước, về sau
    + Có người ngồi, hoặc không có người ngồi
    + Màu sắc
    + Chủng loại - ví dụ Toyota Vios, Toyota Innova, Toyota Corolla Cross
- Thư mục upload: https://drive.google.com/drive/u/1/folders/1Uj0V9URNHpzSHeXHSB89AoGCjGki8Yra
- Deadline ??/??/2024
- Các nhóm sẽ được cung cấp các tập tin gồm:
    + train.txt - danh sách các ảnh dùng cho trainining
    + test.txt - danh sách các ảnh dùng cho testing

## Code

- Các nhóm sẽ được cung cấp notebook về khung của chương trình trong đó có các chức năng gồm
    + train: đọc thông tin từ file train.txt và thực hiện training để output ra model.
    + test: đọc thông tin từ file test.txt và thực hiện prediction bằng cách load model lên và output ra category/label thương ứng.
- Hệ thống sẽ tự động tính performance và hiển thị lên ScoreBoard
- Thư mục để cập nhật code: sẽ thông báo sau



## ScoreBoard

- Tiếp tục cập nhật

## Thảo luận

- Các SV tham gia nhóm Zalo của lớp CS114.P11 để có thông tin cập nhật

# **Thông tin của tác giả, ngày cập nhật**

<hr>

**Thành viên nhóm**:
- **Trần Đình Khánh Đăng - 22520195**
- **Tăng Nhất - 22521027**
- **Lê Minh Nhựt - 22521060**

**Ngày cập nhật**: 22/01/2025

# **PHẦN THỰC NGHIỆM**

## **Install và Import thư viện cần thiết**

In [ ]:
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    "cudf-cu12==24.12.*" "dask-cudf-cu12==24.12.*" "cuml-cu12==24.12.*" \
    "cugraph-cu12==24.12.*" "nx-cugraph-cu12==24.12.*" "cuspatial-cu12==24.12.*" \
    "cuproj-cu12==24.12.*" "cuxfilter-cu12==24.12.*" "cucim-cu12==24.12.*" \
    "pylibraft-cu12==24.12.*" "raft-dask-cu12==24.12.*" "cuvs-cu12==24.12.*" \
    "nx-cugraph-cu12==24.12.*"

In [ ]:
import os
import gc
import re
import cv2
import sys
import csv
import time
import math
import random
import warnings
from collections import defaultdict

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib import patches

from tqdm.notebook import tqdm


from cuml.svm import SVC as cuSVC
from cuml.ensemble import RandomForestClassifier as cuRFClassifier
from cuml.neighbors import KNeighborsClassifier as cuKNNClassifier

from skimage.io import imread
from skimage.color import rgb2gray
from skimage.metrics import structural_similarity as ssim

from PIL import Image
from imagehash import phash
from scipy.spatial.distance import cdist, euclidean

from sklearn.model_selection import KFold
from sklearn.cluster import (KMeans, DBSCAN, AgglomerativeClustering, 
                             MeanShift, Birch, SpectralClustering)
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import learning_curve

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.applications import (MobileNet, MobileNetV2, MobileNetV3Small, MobileNetV3Large, 
                                           ResNet50, ResNet101, ResNet152,
                                           VGG16, VGG19,
                                           EfficientNetB0, EfficientNetB1, EfficientNetB7,
                                           InceptionV3, Xception)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

import plotly.express as px

## **Khởi tạo đường dẫn**

In [ ]:
dataset_dir = '/kaggle/working/dataset'

cropped_file_name = 'cropped_dataset.csv'
cropped_dropdup_file_name = 'dropdup_dataset.csv'

categories = ['Others', 'Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Suzuki', 'Toyota', 'VinFast']

base_dir = '/kaggle/input/cs114-final-project-full-dataset'
dataset_name = 'CarDataset.csv'
file_name_cars = 'CarDataset-1.csv'
file_name_categories = 'CarDataset-2.csv'
extracted_features_file_name='extracted_features.npz'

cropped_base_dir= '/kaggle/input/cs114-cropped-full-dataset/dataset'
cropped_dataset_name = 'cropped_CarDataset.csv'
cropped_file_name_cars = 'cropped_CarDataset-1.csv'
cropped_file_name_categories = 'cropped_CarDataset-2.csv'
cropped_extracted_features_file_name='cropped_extracted_features.npz'

cropped_dropdup_base_dir = '/kaggle/input/cs114-cropped-full-dataset/dataset'
cropped_dropdup_dataset_name = 'cropped_dropdup_CarDataset.csv'
cropped_dropdup_file_name_cars = 'cropped_dropdup_CarDataset-1.csv'
cropped_dropdup_file_name_categories = 'cropped_dropdup_CarDataset-2.csv'
cropped_dropdup_extracted_features_file_name='dropdup_extracted_features.npz'
cropped_dropdup_extracted_features_csv = 'dropdup_extracted_features.csv'

augmented_base_dir='/kaggle/input/cs114-augmented-dataset/augmented_images'
augmented_dataset_name = 'augmented_CarDataset.csv'
augmented_file_name_cars = 'augmented_CarDataset-1.csv'
augmented_file_name_categories = 'augmented_CarDataset-2.csv'
augmented_extracted_features_file_name='augmented_extracted_features.npz'

full_augmented_base_dir='/kaggle/input/cs114-full-augmented-dataset/full_augmented_dataset'
full_augmented_dataset_name = 'full_augmented_CarDataset.csv'
full_augmented_file_name_cars = 'full_augmented_CarDataset-1.csv'
full_augmented_file_name_categories = 'full_augmented_CarDataset-2.csv'
full_augmented_extracted_features_file_name='full_augmented_extracted_features.npz'

def get_indexing(categories):
    indexing = {category: idx for idx, category in enumerate(categories)}
    invert_indexing = {idx: category for category, idx in indexing.items()}
    return indexing, invert_indexing

indexing, invert_indexing = get_indexing(categories)

num_splits = 5

In [ ]:
def create_dataframe(data, sort=False, sort_value=None) -> pd.DataFrame:
    
    df = pd.DataFrame(data)
    if sort and sort_value:
        df = df.sort_values(by=sort_value, ascending=False).reset_index(drop=True)
    return df

def get_image_count(base_dir='./',
                   categories=['Others', 'Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Suzuki', 'Toyota', 'VinFast'],
                   take_average=False,
                   ) -> defaultdict:
    
    # Tạo biến tạm để lưu số lượng ảnh theo MSSV
    image_count = defaultdict(lambda: defaultdict(float))  # Có tình trạng có MSSV không đóng góp ảnh (?)

    student_ids_pattern = r'(\d{8}(?:-\d{8})*)' # Lấy MSSV hợp lệ (đủ 8 số)
    categories_pattern = '|'.join(categories) # Lấy hiệu xe hợp lệ
    file_extension_pattern = r'\.\d+\.(jpg|jpeg|png)$' # Lấy extension hợp lệ (chỉ chấp nhận file .jpg, .jpeg và .png)
    # Regex lấy tên file hợp lệ
    accepted_filename = re.compile(fr'{student_ids_pattern}\.({categories_pattern}){file_extension_pattern}')

    # accepted_filename_2 = re.compile(fr'(\d{{8}}(?:-\d{{8}})*)\.({categories_pattern})\.\d+\.(jpg|jpeg|png)$')
    for category in tqdm(categories, desc="Processing categories"): # Duyệt qua các hiệu xe
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path): # Kiểm tra nếu thư mục tồn tại
            for filename in os.listdir(category_path):
                match = accepted_filename.match(filename)
                if match: # Chỉ xử lý file có tên hợp lệ
                    student_ids, car_category, img_idx = match.groups()
                    student_ids_list = student_ids.split('-')
                    num_members = len(student_ids_list)
                    # Đếm ảnh theo MSSV
                    for student_id in student_ids_list:
                        image_count[student_id][car_category] += 1 / num_members if take_average else 1

    return image_count

def write_csv(base_dir='./',
              dataset_dir='./',
              categories=['Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Others', 'Suzuki', 'Toyota', 'VinFast'],
              file_name_cars='CarDataset-1.csv',
              file_name_categories='CarDataset-2.csv',
              save_csv=False,
              take_average=False,
              sort=False
              ) -> tuple:
    
    os.makedirs(dataset_dir, exist_ok=True) # Tạo thư mục đầu ra nếu chưa tồn tại
    image_count = get_image_count(base_dir=base_dir, categories=categories, take_average=take_average) # Lấy số lượng ảnh theo MSSV và loại xe
    cars_list = [] # Danh sách tổng số lượng ảnh của mỗi MSSV
    categories_list = [] # Danh sách số lượng ảnh theo hiệu xe của mỗi MSSV

    for student_id, car_data in image_count.items():
        total_images = 0
        for car_category, count in car_data.items():
            total_images += count
            categories_list.append({'MSSV': student_id, 'Hiệu xe': car_category, 'Số lượng': round(count, 2)})
        cars_list.append({'MSSV': student_id, 'All': 'All', 'Số lượng': round(total_images, 2)})

    # Chuyển danh sách thành DataFrame
    df_cars = create_dataframe(cars_list, sort=sort, sort_value='Số lượng')
    df_categories = create_dataframe(categories_list, sort=sort, sort_value='Số lượng')

    # Lấy đường dẫn file CSV
    file_path_categories = os.path.join(dataset_dir, file_name_categories)
    file_path_cars = os.path.join(dataset_dir, file_name_cars)

    # Lưu CSV nếu cần
    if save_csv:
        df_categories.to_csv(file_path_categories, index=False)
        df_cars.to_csv(file_path_cars, index=False)

        print(f'{file_name_cars} saved to: {file_path_cars}')
        print(f'{file_name_categories} saved to: {file_path_categories}')

    return df_cars, df_categories


In [ ]:
df_cars, df_categories = write_csv(base_dir=base_dir,
                                   dataset_dir=dataset_dir,
                                   categories=categories,
                                   file_name_cars=file_name_cars,
                                   file_name_categories=file_name_categories,
                                   save_csv=True,
                                   take_average=False,
                                   sort=True)
print(f'{file_name_cars}:\n{df_cars}')
print(f'{file_name_categories}:\n{df_categories}')

## Visualize dữ liệu

In [ ]:
# Nhóm dữ liệu theo 'Hiệu xe' và tính tổng 'Số lượng'
brand_summary = df_categories.groupby('Hiệu xe')['Số lượng'].sum().reset_index()

# Sắp xếp theo số lượng giảm dần để dễ quan sát
brand_summary = brand_summary.sort_values('Số lượng', ascending=False)

In [ ]:
# Thiết lập style cho biểu đồ
sns.set_style("whitegrid")

# Tạo histogram
plt.figure(figsize=(10, 6))
sns.histplot(df_cars['Số lượng'], bins=30, kde=True)
plt.title('Phân phối tổng số lượng xe đóng góp theo sinh viên')
plt.xlabel('Tổng số lượng xe')
plt.ylabel('Số lượng sinh viên')
plt.tight_layout()
plt.show()

In [ ]:
brand_summary.sort_values('Số lượng', ascending=True, inplace=True)
plt.figure(figsize=(12, 6))
sns.barplot(x='Hiệu xe', y='Số lượng', data=brand_summary)
plt.title('Tổng số lượng xe theo hiệu xe')
plt.xlabel('Hiệu xe')
plt.ylabel('Tổng số lượng xe')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
colors = sns.color_palette('pastel')[0:len(brand_summary)]

plt.figure(figsize=(8, 8))
plt.pie(brand_summary['Số lượng'], labels=brand_summary['Hiệu xe'], autopct='%1.1f%%', startangle=140, colors=colors)
plt.title('Tỷ lệ phần trăm số lượng xe theo hiệu xe')
plt.show()

In [ ]:
def get_dataset(base_dir='./',
                dataset_dir='/',
                categories=['Others', 'Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Suzuki', 'Toyota', 'VinFast'],
                save_csv=False,
                file_name='CarDataset.csv',
                ) -> pd.DataFrame:
    
    os.makedirs(dataset_dir, exist_ok=True)  # Tạo thư mục đầu ra nếu chưa tồn tại

    path_list = []  # Lưu đường dẫn đầy đủ của hình ảnh
    categoryid_list = []  # Lưu mã danh mục tương ứng với từng hình ảnh


    student_ids_pattern = r'(\d{8}(?:-\d{8})*)' # Lấy MSSV hợp lệ (đủ 8 số)
    categories_pattern = '|'.join(categories) # Lấy hiệu xe hợp lệ
    file_extension_pattern = r'\.\d+\.(jpg|jpeg|png)$' # Lấy extension hợp lệ (chỉ chấp nhận file .jpg, .jpeg và .png)
    # Regex lấy tên file hợp lệ
    accepted_filename = re.compile(fr'{student_ids_pattern}\.({categories_pattern}){file_extension_pattern}')

    for category in tqdm(categories, desc="Processing categories"): # Duyệt qua các hiệu xe
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path): # Kiểm tra nếu thư mục tồn tại
            for filename in os.listdir(category_path):
                match = accepted_filename.match(filename)
                if match: # Chỉ xử lý file có tên hợp lệ
                    _, car_category, _ = match.groups()
                    if car_category in categories:
                        full_path = os.path.join(car_category, filename)
                        path_list.append(full_path)
                        categoryid_list.append(indexing[car_category])
    # Tạo DataFrame từ danh sách đường dẫn và mã hiệu xe
    df = pd.DataFrame({
        'ImageFullPath': path_list,
        'CategoryID': categoryid_list
    })

    # Lưu CSV nếu cần
    if save_csv:
        output_file = os.path.join(dataset_dir, file_name)
        df.to_csv(output_file, index=False)
        print(f"{file_name} saved to {output_file}")

    return df

In [ ]:
df = get_dataset(base_dir=base_dir,
                 dataset_dir=dataset_dir,
                 categories=categories,
                 save_csv=True)

# Bước 1: Dùng object detection để crop ảnh

In [ ]:
model_url = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"
detector = hub.load(model_url)

def save_cropped_image(cropped_image, original_image_path, dataset_dir='./'):
    base_dir, img_name = os.path.split(original_image_path)
    class_name = os.path.basename(os.path.dirname(original_image_path))
    new_dir = os.path.join(dataset_dir, class_name)
    os.makedirs(new_dir, exist_ok=True)
    cropped_image_path = os.path.join(new_dir, img_name)
    cv2.imwrite(cropped_image_path, cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR))
    return cropped_image_path

def load_and_preprocess_image(base_dir, image_path, max_image_size):
    image_path = os.path.join(base_dir, image_path)
    img = cv2.imread(image_path)
    if img is None:
        print(f"Cannot load image from {image_path}")
        return None, None
    
    if max_image_size is not None:
        img = cv2.resize(img, max_image_size)
    
    original_img = img.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img, original_img

def detect_car(image, detector, score_threshold=0.5):
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = tf.expand_dims(input_tensor, axis=0)
    detections = detector.signatures['serving_default'](input_tensor)

    boxes = detections['detection_boxes'].numpy()[0]
    class_ids = detections['detection_classes'].numpy()[0].astype(np.int64)
    scores = detections['detection_scores'].numpy()[0]

    return {"boxes": boxes, "class_ids": class_ids, "scores": scores}

def process_chunk(data_chunk, base_dir, dataset_dir, detector, class_index, score_threshold, save_images, none_list_file, max_image_size):
    batch_results = []
    for idx, row in tqdm(data_chunk.iterrows(), total=data_chunk.shape[0]):
        image_path = row['ImageFullPath']
        image_np, original_image = load_and_preprocess_image(base_dir, image_path, max_image_size)

        if image_np is None:
            batch_results.append((image_path, None, None, None))
            continue

        detection_results = detect_car(image_np, detector, score_threshold)

        boxes = detection_results["boxes"]
        class_ids = detection_results["class_ids"]
        scores = detection_results["scores"]

        img_h, img_w, _ = original_image.shape

        max_area = 0
        prioritized_box = None
        best_score = 0

        for box, score, label in zip(boxes, scores, class_ids):
            if score < score_threshold or label != class_index:
                continue
            ymin, xmin, ymax, xmax = box
            area = (xmax - xmin) * (ymax - ymin)

            if area > max_area:
                max_area = area
                prioritized_box = (xmin, ymin, xmax, ymax)
                best_score = score

        cropped_image_path = None
        prioritized_box_pixels = None

        if prioritized_box is not None:
            xmin, ymin, xmax, ymax = prioritized_box
            left = int(xmin * img_w)
            right = int(xmax * img_w)
            top = int(ymin * img_h)
            bottom = int(ymax * img_h)

            cropped_image = original_image[top:bottom, left:right]
            if save_images:
                cropped_image_path = save_cropped_image(cropped_image, image_path, dataset_dir)
            prioritized_box_pixels = (left, right, top, bottom)
        else:
            if save_images:
                cropped_image = original_image 
                cropped_image_path = save_cropped_image(cropped_image, image_path, dataset_dir)
            with open(os.path.join(dataset_dir, none_list_file), 'a') as f:
                f.write(f"{image_path}, No_Car_Detected\n")

        batch_results.append((image_path, cropped_image_path, prioritized_box_pixels, best_score))

    return batch_results

def process_car_detection(base_dir='./',
                          dataset_dir='./',
                          data=None,
                          file_name='CarDataset.csv',
                          detector=None,
                          partition=False,
                          partition_size=1000,
                          batch_size=10,
                          chunk_size=100,  # Chunk size for reading CSV
                          class_index=3,
                          score_threshold=0.5,
                          save_images=True,
                          none_list_file='NoneList.csv',
                          max_image_size=(640, 640)
                          ):
    data_path = os.path.join(dataset_dir, file_name)
    results_list = []

    total_rows = sum(1 for row in open(data_path)) -1 
    num_chunks = math.ceil(total_rows / chunk_size)

    print(f"Total number of chunks: {num_chunks}")

    chunk_num = 0 
    for chunk in pd.read_csv(data_path, chunksize=chunk_size):
        chunk_num += 1
        print(f"Processing chunk {chunk_num} out of {num_chunks} (Size: {len(chunk)} rows)")

        if partition:
            chunk = chunk.sample(n=min(partition_size, len(chunk)), random_state=42).reset_index(drop=True)
            print(f"  - Processing {len(chunk)} images after partitioning...")

        total_batches = math.ceil(len(chunk) / batch_size)
        print(f"  - Total batches in this chunk: {total_batches}")

        for i in range(0, len(chunk), batch_size):
            batch_data = chunk[i:i + batch_size]
            batch_results = process_chunk(batch_data, base_dir, dataset_dir, detector, class_index, score_threshold, save_images, none_list_file, max_image_size)
            results_list.extend(batch_results)

            del batch_data
            del batch_results
            gc.collect()

    return results_list


In [ ]:
results_list = process_car_detection(base_dir=base_dir,
                                     dataset_dir=dataset_dir,
                                     data=df,
                                     detector=detector,
                                     batch_size = 128,
                                     chunk_size = 1024,
                                     )

## **Visualize Kết quả**

In [ ]:
def visualize_results(results_list,
                      data=None,
                      base_dir='./',
                      dataset_dir='./',
                      file_name='CarDataset.csv',
                      partition=False,
                      partition_size=10,
                      ):
    
    if data is None:
        data_path = os.path.join(dataset_dir, file_name)
        data = pd.read_csv(data_path)
    if partition:
        start_index = (partition - 1) * partition_size
        end_index = min(start_index + partition_size, len(results_list))
        results_list = results_list[start_index:end_index]

    for i, (image_path, cropped_image_path, bbox, score) in tqdm(enumerate(results_list), total=len(results_list), desc="Processing images: "):
        image_path = os.path.join(base_dir, image_path)
        try:
            original_image = cv2.imread(image_path)
            original_image = cv2.resize(original_image, (640, 640))
            original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
            if original_image is None:
                print(f"Warning: Could not load image at {image_path}. Skipping.")
                continue
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            continue

        fig, ax = plt.subplots(1, 2, figsize=(15, 5))

        # Turn off the grid for both subplots
        ax[0].grid(False)
        ax[1].grid(False)

        ax[0].imshow(original_image)
        ax[0].set_title(f"Original Image: {os.path.basename(image_path)}")

        if bbox:
            left, right, top, bottom = bbox
            rect = patches.Rectangle((left, top), right - left, bottom - top,
                                     linewidth=2, edgecolor='r', facecolor='none')
            ax[0].add_patch(rect)
            ax[0].text(left, top - 5, f"Car (Score: {score:.2f})", color='red', fontsize=10)

        if cropped_image_path is not None:
            cropped_image = cv2.imread(cropped_image_path)
            ax[1].imshow(cropped_image)
            ax[1].set_title("Cropped Car")
        else:
            ax[1].set_title("No Car Detected Above Threshold")

        plt.tight_layout()
        plt.show()
        plt.close()

In [ ]:
visualize_results(base_dir=base_dir,
                  dataset_dir = dataset_dir,
                    results_list = results_list,
                    data=df,
                    partition=True,
                    partition_size=10)

# Bước 2: Loại bỏ ảnh trùng

In [ ]:
def extract_feature_one_img(image_path, model, input_shape=(224, 224)):
    img = image.load_img(image_path, target_size=input_shape)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x, verbose=0)
    return features.flatten()

def extract_features(data=None,
                     base_dir='./',
                     dataset_dir='./',
                     file_csv='CarDataset-Splits-1-Train.csv',
                     model_name='MobileNet',
                     input_shape=(224, 224),
                     partition=False,
                     partition_size=1000,
                     random_state=42,
                     save_result=False,
                     save_name='extracted_features-Splits-1.npz'):
    models = {
        'MobileNet': MobileNet,
        'MobileNetV2': MobileNetV2,
        'MobileNetV3Small': MobileNetV3Small,
        'MobileNetV3Large': MobileNetV3Large,
        'ResNet50': ResNet50,
        'ResNet101': ResNet101,
        'ResNet152': ResNet152,
        'VGG16': VGG16,
        'VGG19': VGG19,
        'EfficientNetB0': EfficientNetB0,
        'EfficientNetB1': EfficientNetB1,
        'EfficientNetB7': EfficientNetB7,
        'InceptionV3': InceptionV3,
        'Xception': Xception
    }

    if model_name not in models:
        model_name = 'MobileNet'

    device = '/device:GPU:0' if tf.config.list_physical_devices('GPU') else '/device:CPU:0'
    with tf.device(device):
        model = models[model_name](weights='imagenet', include_top=False, pooling='avg')

    if data is None:
        data = pd.read_csv(os.path.join(dataset_dir, file_csv))

    if partition:
        sampled_data = data.sample(n=min(partition_size, len(data)), random_state=random_state).reset_index(drop=True)
        print(f"Processing {len(sampled_data)} images out of {len(data)} available.")
    else:
        sampled_data = data
    print("Extracting features...")

    result = []
    for _, row in tqdm(sampled_data.iterrows(), desc="Extracting Features", total=len(sampled_data), file=sys.stdout, leave=True):
        image_path = row["ImageFullPath"]
        categoryid = row["CategoryID"]

        full_path = os.path.join(base_dir, image_path)
        try:
            extracted_features = extract_feature_one_img(full_path, model, input_shape=input_shape)
            result.append({'ImageFullPath': image_path, 'CategoryID': categoryid, 'Extracted Features': extracted_features})
        except Exception as e:
            print(f"Error processing image {full_path}: {e}. Skipping...")

    print(f"Successfully processed {len(result)} images")

    if save_result:
        save_path = os.path.join(dataset_dir, save_name)
        np.savez(save_path, extracted_features=result)
        print(f"Extracted features saved to {save_path}")

    return result

def load_features(file_path):
    try:
        # Tải file .npz và lấy dữ liệu 'extracted_features'
        data = np.load(file_path, allow_pickle=True)
        extracted_features = data['extracted_features']

        # Đảm bảo định dạng giống như khi lưu
        formatted_features = []
        for item in extracted_features:
            # Chuyển từng phần tử từ ndarray về dictionary với đúng format
            formatted_features.append({
                'ImageFullPath': item['ImageFullPath'],
                'CategoryID': item['CategoryID'],
                'Extracted Features': item['Extracted Features']
            })

        print(f"Loaded extracted features from {file_path}")
        return formatted_features
    except Exception as e:
        print(f"Error loading features from {file_path}: {e}")
        return None

In [ ]:
df_cropped = get_dataset(base_dir=cropped_base_dir,
                         dataset_dir=dataset_dir,
                         categories=categories,
                         file_name=cropped_file_name,
                         save_csv=True)

In [ ]:
# st = time.time()
# extracted_features = extract_features(base_dir=cropped_base_dir,
#                                       dataset_dir=dataset_dir,
#                                       file_csv=cropped_file_name,
#                                       model_name='XceptionNet',
#                                       input_shape=(224, 224),
#                                       partition=False,
#                                       partition_size=1000,
#                                       save_result=True,
#                                       save_name=extracted_features_file_name
#                                       )
# print(f"Total time: {(time.time()-st)/3600:.2f}h")
# Load extracted_features đã có từ trước
extracted_features = load_features('/kaggle/input/cs114-cropped-full-dataset/dataset/cropped_extracted_features.npz')
df_extracted_features = pd.DataFrame([feature for feature in tqdm(extracted_features, desc="Turning to DataFrame")])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def to_grayscale(features_list):
    processed_features = []
    for feature in features_list:
        if feature.ndim == 3:
            gray_feature = rgb2gray(feature)
        else:
            gray_feature = feature
        processed_features.append(gray_feature)
    return np.array(processed_features)

def find_duplicate(extracted_features=None,
                   base_dir='./',  # Used for 'ssim'
                   dataset_dir='./',
                   extracted_file='extracted_features.npz',
                   similarity_threshold=0.9,
                   metric='cosine',
                   save_result=False,
                   save_name='duplicate_images.csv'):

    if extracted_features is None:
        file_path = os.path.join(dataset_dir, extracted_file)
        data = np.load(file_path, allow_pickle=True)
        extracted_features = data['extracted_features']

    features_list = [x['Extracted Features'] for x in extracted_features]
    image_path_list = [x['ImageFullPath'] for x in extracted_features]
    categories_list = [x['CategoryID'] for x in extracted_features]
    features_list = to_grayscale(features_list)

    features_tensor = torch.tensor(np.array(features_list), device=device)

    if metric == 'phash':
        precomputed_hashes = {img_path: phash(Image.open(os.path.join(base_dir, img_path)))
                              for img_path in tqdm(image_path_list, desc='Computing hashes', unit='image')}

    duplicate_images = []
    duplicate_indices = []
    num_images = len(features_list)

    for i in tqdm(range(num_images), desc="Processing images", unit="image"):
        # Get the current feature and move it to GPU
        current_feature = features_tensor[i].unsqueeze(0)  # Add batch dimension

        # Compute similarities in batches
        if metric == 'cosine':
            # Compute cosine similarity using PyTorch
            similarities = torch.nn.functional.cosine_similarity(
                current_feature, features_tensor[i + 1:], dim=1
            )
            # Find indices where similarity > threshold
            duplicate_mask = similarities > similarity_threshold
            duplicate_indices_batch = torch.nonzero(duplicate_mask).squeeze(1) + i + 1

        elif metric == 'euclidean':
            # Compute Euclidean distance using PyTorch
            distances = torch.cdist(current_feature, features_tensor[i + 1:], p=2).squeeze(0)
            # Find indices where distance < (1 - threshold)
            duplicate_mask = distances < (1.0 - similarity_threshold)
            duplicate_indices_batch = torch.nonzero(duplicate_mask).squeeze(1) + i + 1

        elif metric == 'ssim':
            # SSIM is not easily parallelizable, so we fall back to CPU
            for j in range(i + 1, num_images):
                similarity = ssim(features_list[i].reshape(-1), features_list[j].reshape(-1), data_range=1)
                if similarity > similarity_threshold:
                    print(f"Found duplicate images: {image_path_list[i]} and {image_path_list[j]} with SSIM {similarity}")
                    duplicate_images.append((image_path_list[i], image_path_list[j], similarity))
                    duplicate_indices.append((i, j))

        elif metric == 'phash':
            # pHash is computed on CPU
            hash1 = precomputed_hashes[image_path_list[i]]
            for j in range(i + 1, num_images):
                hash2 = precomputed_hashes[image_path_list[j]]
                similarity = 1 - (hash1 - hash2) / len(hash1.hash)
                if similarity > similarity_threshold:
                    duplicate_images.append((image_path_list[i], image_path_list[j], similarity))
                    duplicate_indices.append((i, j))
                    print(f"Found duplicate images: {image_path_list[i]} and {image_path_list[j]} with SSIM {similarity}")

        # For cosine and Euclidean, process the batch results
        if metric in ['cosine', 'euclidean']:
            for j in duplicate_indices_batch.cpu().numpy():
                if metric == 'cosine':
                    similarity = similarities[j - i - 1].item()
                    print(f"Found duplicate images: {image_path_list[i]} and {image_path_list[j]} with cosine similarity {similarity}")
                elif metric == 'euclidean':
                    distance = distances[j - i - 1].item()
                    print(f"Found duplicate images: {image_path_list[i]} and {image_path_list[j]} with Euclidean distance {distance}")
                duplicate_images.append((image_path_list[i], image_path_list[j], similarity if metric == 'cosine' else distance))
                duplicate_indices.append((i, j))

    if save_result:
        save_path = os.path.join(dataset_dir, save_name)
        if metric in ['cosine', 'ssim', 'phash']:
            df = pd.DataFrame(duplicate_images, columns=["Image1", "Image2", "Score"])
        else:
            df = pd.DataFrame(duplicate_images, columns=["Image1", "Image2", "Distance"])
        df.to_csv(save_path, index=False)
        print(f"Saved to {save_path}")

    return duplicate_images, duplicate_indices

In [ ]:
# duplicate_images = find_duplicate(extracted_features=extracted_features,
#                                   base_dir=base_dir,
#                                   dataset_dir=dataset_dir,
#                                   save_result = True,
#                                   metric='cosine') # Xịn nhất nhưng lâu nhất
# duplicate_images_2 = find_duplicate(extracted_features=extracted_features,
#                                   base_dir=base_dir,
#                                   dataset_dir=dataset_dir,
#                                   save_result = True,
#                                   metric='ssim') # Tàm tạm, không xịn
# duplicate_images_3, duplicate_indices_3 = find_duplicate(extracted_features=extracted_features[:10000],
#                                   base_dir=base_dir,
#                                   dataset_dir=dataset_dir,
#                                   save_result = True,
#                                   metric='euclidean') # Phế, nhưng nhanh
# duplicate_images_4 = find_duplicate(extracted_features=extracted_features,
#                                   base_dir=base_dir,
#                                   dataset_dir=dataset_dir,
#                                   save_result = True,
#                                   metric='phash') # Ổn

## Tiến hành loại bỏ ảnh trùng

In [ ]:
all_duplicate_images = []
all_duplicate_indices = []

# Automate processing in chunks of 1000
num_images = len(extracted_features)
chunk_size = 36740

for i in tqdm(range(0, num_images, chunk_size), desc='Processing Batches', unit='batch'):
    start_idx = i
    end_idx = min(i + chunk_size, num_images)  # Ensure the last chunk doesn't exceed the list
    chunk = extracted_features[start_idx:end_idx]
    
    # Find duplicates in the current chunk
    duplicate_images, duplicate_indices = find_duplicate(
        extracted_features=chunk,
        base_dir=cropped_base_dir,
        dataset_dir=dataset_dir,
        save_result=True,
        save_name=f'duplicate_images_{start_idx + 1}_{end_idx}.csv',
        metric='cosine',
        similarity_threshold=0.95
    )
    
    # Append the results to the combined lists
    all_duplicate_images.extend(duplicate_images)
    all_duplicate_indices.extend(duplicate_indices)
    
    print(f"Processed images {start_idx + 1} to {end_idx}")

# Create a combined DataFrame
df_combined = pd.DataFrame(all_duplicate_images, columns=["Image1", "Image2", "Distance"])

# Add the indices to the DataFrame
df_combined['Index1'] = [idx[0] for idx in all_duplicate_indices]
df_combined['Index2'] = [idx[1] for idx in all_duplicate_indices]

# Save the combined results to a CSV file
combined_save_path = os.path.join(dataset_dir, "cosine_combined_duplicate_images.csv")
df_combined.to_csv(combined_save_path, index=False)
print(f"Combined duplicate image results saved to {combined_save_path}")

In [ ]:
all_duplicate_images = []
all_duplicate_indices = []

# Automate processing in chunks of 1000
num_images = len(extracted_features)
chunk_size = 36740

for i in tqdm(range(0, num_images, chunk_size), desc='Processing Batches', unit='batch'):
    start_idx = i
    end_idx = min(i + chunk_size, num_images)  # Ensure the last chunk doesn't exceed the list
    chunk = extracted_features[start_idx:end_idx]
    
    # Find duplicates in the current chunk
    duplicate_images, duplicate_indices = find_duplicate(
        extracted_features=chunk,
        base_dir=cropped_base_dir,
        dataset_dir=dataset_dir,
        save_result=True,
        save_name=f'duplicate_images_{start_idx + 1}_{end_idx}.csv',
        metric='euclidean',
        similarity_threshold=0.95
    )
    
    # Append the results to the combined lists
    all_duplicate_images.extend(duplicate_images)
    all_duplicate_indices.extend(duplicate_indices)
    
    print(f"Processed images {start_idx + 1} to {end_idx}")

# Create a combined DataFrame
df_combined = pd.DataFrame(all_duplicate_images, columns=["Image1", "Image2", "Distance"])

# Add the indices to the DataFrame
df_combined['Index1'] = [idx[0] for idx in all_duplicate_indices]
df_combined['Index2'] = [idx[1] for idx in all_duplicate_indices]

# Save the combined results to a CSV file
combined_save_path = os.path.join(dataset_dir, "euclidean_combined_duplicate_images.csv")
df_combined.to_csv(combined_save_path, index=False)
print(f"Combined duplicate image results saved to {combined_save_path}")

In [ ]:
def remove_duplicates(df_extracted_features,
                      duplicate_indices,
                      removal_strategy='keep_first',
                      dataset_dir='./',
                      save_results=False,
                      extracted_features_name='extracted_features.npz',
                      extracted_dataframe_name='dropdup_extracted_features.csv'):
    indices_to_remove = set()
    removed_image_paths = []  # List to store removed image paths

    for i, j in duplicate_indices:
        if removal_strategy == 'keep_first':
            indices_to_remove.add(j)
            removed_image_paths.append(df_extracted_features['ImageFullPath'].iloc[j])  # Add removed image path
        elif removal_strategy == 'keep_second':
            indices_to_remove.add(i)
            removed_image_paths.append(df_extracted_features['ImageFullPath'].iloc[i])  # Add removed image path
        elif removal_strategy == 'keep_smaller':
            if 'ImageFullPath' not in df_extracted_features.columns:
                print("Warning: 'ImageFullPath' column not found. Defaulting to 'keep_first'.")
                indices_to_remove.add(j)
                removed_image_paths.append(df_extracted_features['ImageFullPath'].iloc[j])  # Add removed image path
            else:
                path_i = os.path.join(dataset_dir, df_extracted_features['ImageFullPath'].iloc[i])
                path_j = os.path.join(dataset_dir, df_extracted_features['ImageFullPath'].iloc[j])
                try:
                    size_i = os.path.getsize(path_i)
                    size_j = os.path.getsize(path_j)

                    if size_i <= size_j:
                        indices_to_remove.add(j)
                        removed_image_paths.append(df_extracted_features['ImageFullPath'].iloc[j])  # Add removed image path
                    else:
                        indices_to_remove.add(i)
                        removed_image_paths.append(df_extracted_features['ImageFullPath'].iloc[i])  # Add removed image path
                except FileNotFoundError:
                    print(f"Warning: One or both files not found: {path_i}, {path_j}. Defaulting to 'keep_first'.")
                    indices_to_remove.add(j)
                    removed_image_paths.append(df_extracted_features['ImageFullPath'].iloc[j])  # Add removed image path
        else:
            raise ValueError(f"Invalid removal_strategy: {removal_strategy}")

    indices_to_remove = sorted(
        [df_extracted_features.index[idx] for idx in indices_to_remove if idx < len(df_extracted_features.index)],
        reverse=True
    )
    new_df_extracted_features = df_extracted_features.drop(indices_to_remove)

    extracted_features_list = []
    # Duyet qua tung dong dataframe
    for _, row in tqdm(new_df_extracted_features.iterrows(), desc="Extracting Features", total=len(new_df_extracted_features), file=sys.stdout, leave=True):
        image_path = row["ImageFullPath"]
        categoryid = row["CategoryID"]
        extracted_feature = row["Extracted Features"]
        # Lay duong dan full
        full_path = os.path.join(dataset_dir, image_path)
        try:
            extracted_features_list.append({'ImageFullPath': image_path, 'CategoryID': categoryid, 'Extracted Features': extracted_feature})
        except Exception as e:
            print(f"Error at image {full_path}: {e}. Skipping...")

    print(f"Successfully processed {len(extracted_features_list)} images")
    new_df_extracted_features['Extracted Features'] = [entry['Extracted Features'] for entry in extracted_features_list]
    new_extracted_features = new_df_extracted_features.reset_index().drop('index', axis=1)
    if save_results:
        df_file_path = os.path.join(dataset_dir, extracted_dataframe_name)
        features_file_path = os.path.join(dataset_dir, extracted_features_name)

        new_df_extracted_features.to_csv(df_file_path, index=False)
        print(f"Saved DataFrame to: {df_file_path}")

        np.savez(features_file_path, extracted_features=extracted_features_list)
        print(f"Saved extracted features to: {features_file_path}")

    return new_df_extracted_features, extracted_features_list, removed_image_paths  # Return removed image paths

In [ ]:
new_df_extracted_features, extracted_features_list, indices_to_remove = remove_duplicates(
    df_extracted_features=df_extracted_features,
    duplicate_indices=all_duplicate_indices,
    removal_strategy='keep_first', 
    dataset_dir=dataset_dir,
    save_results=True,
    extracted_features_name=cropped_dropdup_extracted_features_file_name,
    extracted_dataframe_name=cropped_dropdup_extracted_features_csv
)

## **Visualize Kết quả**

In [ ]:
def plot_duplicate_images(image_pairs, base_dir):
    rows = len(image_pairs)
    fig, axes = plt.subplots(rows, 2, figsize=(10, rows * 3))

    if rows == 1:
        axes = [axes]

    for i, (image_path1, image_path2, scores) in tqdm(enumerate(image_pairs), desc="Displaying images", total=len(image_pairs), unit='image'):
        full_path1 = os.path.join(base_dir, image_path1)
        full_path2 = os.path.join(base_dir, image_path2)

        img1 = Image.open(full_path1)
        img2 = Image.open(full_path2)

        axes[i][0].imshow(img1)
        axes[i][0].axis('off')
        axes[i][0].set_title(image_path1.split('/')[-1])

        axes[i][1].imshow(img2)
        axes[i][1].axis('off')
        axes[i][1].set_title(image_path2.split('/')[-1])

    plt.tight_layout()
    plt.show()

In [ ]:
plot_duplicate_images(all_duplicate_images[:10], base_dir=cropped_base_dir)

# Bước 3: Data Augmentation

## **TorchVision**

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, base_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.base_dir = base_dir
        self.transform = transform

        # Duyệt thư mục lớn để tạo từ điển map từ folder đến ảnh
        self.image_paths = self._find_image_paths()

    def _find_image_paths(self):
        # Tạo từ điển chứa tên folder và đường dẫn đầy đủ tới ảnh trong folder đó
        image_paths = {}
        
        # Use tqdm to show progress while iterating through folders
        for folder_name in tqdm(self.data_frame.iloc[:, 0].values, desc="Scanning folders"):
            folder_path = os.path.join(self.base_dir, folder_name)
            if os.path.isdir(folder_path):  # Kiểm tra nếu folder tồn tại
                for img_file in os.listdir(folder_path):
                    # Thêm ảnh vào từ điển với key là tên folder và value là đường dẫn ảnh
                    img_path = os.path.join(folder_path, img_file)
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        image_paths[img_file] = img_path
        return image_paths

    def __len__(self):
        return len(self.data_frame)  # Corrected from self.data_frames to self.data_frame

    def __getitem__(self, idx):
        # Lấy tên folder từ file CSV
        folder_name = self.data_frame.iloc[idx, 0]

        # Lấy class tương ứng từ file CSV
        label = int(self.data_frame.iloc[idx, 1])

        # Lấy ảnh đầu tiên từ folder tương ứng
        img_path = None
        img_path = os.path.join(self.base_dir, folder_name)  # Corrected from base_dir to self.base_dir

        if img_path is None:
            raise FileNotFoundError(f"Không tìm thấy ảnh trong thư mục {folder_name}")

        # Mở ảnh bằng PIL
        image = Image.open(img_path).convert("RGB")

        # Áp dụng các phép biến đổi ảnh nếu có
        if self.transform:
            image = self.transform(image)

        return image, label, img_path

In [ ]:
csv_file = "/kaggle/input/cs114-extracted-features/dropdup_extracted_features.csv"

# Phép biến đổi ảnh (resize, chuyển thành tensor, normalize)
img_size = 640

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),   # Thay đổi kích thước ảnh
    transforms.ToTensor(),         # Chuyển thành tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize ảnh
])

# Tạo dataset từ file CSV và thư mục ảnh
train_dataset = CustomImageDataset(csv_file=csv_file,
                                   base_dir=cropped_base_dir,
                                   transform=transform)

# Tạo DataLoader để load dữ liệu
dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Hàm hiển thị ảnh với khả năng điều chỉnh kích thước figure
def imshow(img, figsize=(30, 30)):  # figsize là kích thước (width, height) của figure
    # Đảo ngược chuẩn hóa ảnh
    img = img / 2 + 0.5  # Đảo ngược chuẩn hóa (vì đã chuẩn hóa với mean=0.5, std=0.5)
    npimg = img.numpy()  # Chuyển tensor thành numpy
    plt.figure(figsize=figsize)  # Thiết lập kích thước figure
    plt.imshow(np.transpose(npimg, (1, 2, 0)))  # Đảo chiều từ (C, H, W) -> (H, W, C)
    plt.grid(False)
    plt.show()

# Kiểm tra dữ liệu bằng cách in ra 1 batch và hiển thị ảnh
for images, labels, _ in tqdm(dataloader, total=dataloader.__len__(), desc="Processing batches"):  # Thêm tqdm ở đây
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")

    # Hiển thị batch ảnh đầu tiên (với kích thước lớn hơn)
    imshow(torchvision.utils.make_grid(images), figsize=(12, 12))  # Kích thước lớn hơn (width=12, height=12)

    break  # Hiển thị 1 batch thôi

In [ ]:
def pytorch_to_numpy(image_tensor):
    return image_tensor.cpu().permute(1, 2, 0).numpy()  # Chuyển từ (C, H, W) thành (H, W, C)

# Hàm hiển thị ảnh
def plot_images(original_img, augmented_imgs, n_samples=5, figsize=(15, 5)):
    plt.figure(figsize=figsize)

    # Vẽ ảnh gốc ở vị trí đầu tiên
    plt.subplot(1, n_samples + 1, 1)
    plt.imshow(original_img)
    plt.axis('off')
    plt.title("Original Image")

    # Vẽ các ảnh augmented ở các vị trí còn lại
    for i in range(n_samples):
        plt.subplot(1, n_samples + 1, i + 2)
        plt.imshow(augmented_imgs[i])
        plt.axis('off')
        plt.title(f"Augmented {i+1}")

    plt.show()

# Hàm áp dụng data augmentation bằng PyTorch
def apply_augmentations_with_pytorch(dataloader,
                                     n_samples=7, 
                                     output_dir="/kaggle/working/augmented_images", 
                                     plot_images=True,
                                     verbose=True):
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(output_dir, exist_ok=True)

    # Lấy một ảnh mẫu để có kích thước ảnh
    for images, _, img_paths in dataloader:  # Thêm img_paths vào vòng lặp
        img_size = images[0].shape[1]  # Lấy chiều cao hoặc chiều rộng vì ảnh là vuông
        break

    # Khởi tạo các phép biến đổi sử dụng torchvision.transforms
    transform = transforms.Compose([
        transforms.RandomRotation(30),         # Quay ngẫu nhiên ảnh trong phạm vi 30 độ
        transforms.RandomHorizontalFlip(),     # Lật ngang ngẫu nhiên
        transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),  # Cắt và thay đổi kích thước ngẫu nhiên
        transforms.ColorJitter(
            brightness=random.uniform(0, 0.1),
            contrast=random.uniform(0.5, 0.7),
            saturation=random.uniform(0.5, 0.7),
            hue=random.uniform(0, 0.2)
        )  # Điều chỉnh thông số ColorJitter hợp lý hơn
    ])

    # Lặp qua dữ liệu trong dataloader với tqdm
    for batch_idx, (images, _, img_paths) in enumerate(tqdm(dataloader, total=dataloader.__len__(), desc="Processing batches")):
        # Chuyển mỗi batch ảnh từ tensor sang numpy array
        for image_tensor, img_path in zip(images, img_paths):  # Lặp qua từng ảnh và đường dẫn
            # In ra đường dẫn ảnh
            print(f"Image Path: {img_path}") if verbose else None
            image_tensor = image_tensor.to(device)

            # Ảnh gốc
            original_image_np = pytorch_to_numpy(image_tensor)
            original_image_np = (original_image_np * 0.5 + 0.5).clip(0, 1)  # Đảo ngược chuẩn hóa

            # Áp dụng augmentations
            augmented_imgs = []
            for aug_idx in range(n_samples):
                # Áp dụng phép biến đổi cho mỗi ảnh
                augmented_image_tensor = transform(image_tensor * 0.5 + 0.5)
                augmented_image_np = pytorch_to_numpy(augmented_image_tensor)
                augmented_image_np = (augmented_image_np).clip(0, 1)  # Đảm bảo ảnh sau augment nằm trong khoảng [0, 1]
                augmented_imgs.append(augmented_image_np)

                # Lấy tên folder từ đường dẫn ảnh
                folder_name = img_path.split('/')[-2]  # Lấy tên folder từ đường dẫn (ví dụ: "Honda")
                folder_output_dir = os.path.join(output_dir, folder_name)  # Thư mục đầu ra cho folder này
                os.makedirs(folder_output_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

                # Lưu ảnh đã augment
                img_name = os.path.basename(img_path)  # Lấy tên file từ đường dẫn
                img_name_without_ext = os.path.splitext(img_name)[0]  # Bỏ phần mở rộng
                output_path = os.path.join(folder_output_dir, f"{img_name_without_ext}_augmented_{aug_idx + 1}.jpg")
                save_image(augmented_image_tensor.cpu(), output_path)  # Lưu ảnh (chuyển lại về CPU trước khi lưu)

            # Plot ảnh gốc và các ảnh augmented nếu plot_images=True
            if plot_images:
                plot_images(original_image_np, augmented_imgs, n_samples=n_samples)

In [ ]:
apply_augmentations_with_pytorch(dataloader,
                                 n_samples=5,
                                 plot_images=False,
                                 verbose=False)

# Bước 4: Clustering

In [ ]:
def split_kfold_datasets(data=None,
                        num_splits=5,
                        base_dir='./',
                        dataset_dir='./',
                        save_csv=False,
                        file_name='CarDataset.csv',
                        prefix=None,
                        random_state=42,
                        ):
    os.makedirs(dataset_dir, exist_ok=True)  # Tạo thư mục đầu ra nếu chưa tồn tại
    if data == None:
      # Đọc dữ liệu từ file CSV ban đầu
      data_path = os.path.join(dataset_dir, file_name)
      data = pd.read_csv(data_path)

    path_list = data['ImageFullPath'].values # Danh sách đường dẫn hình ảnh
    categoryid_list = data['CategoryID'].values # Danh sách mã hiệu xe tương ứng

    train_splits = []  # Danh sách chứa các DataFrame tập train
    test_splits = []  # Danh sách chứa các DataFrame tập test

    # Khởi tạo đối tượng KFold
    kfold = KFold(n_splits=num_splits, shuffle=True, random_state=random_state)

    for i, (train_index, test_index) in enumerate(kfold.split(path_list)):
        # Đường dẫn file train/test cho từng tập con
        if prefix is not None:
            train_file_path = os.path.join(dataset_dir, f'{prefix}_CarDataset-Splits-{i + 1}-Train.csv')
            test_file_path = os.path.join(dataset_dir, f'{prefix}_CarDataset-Splits-{i + 1}-Test.csv')
        else:
            train_file_path = os.path.join(dataset_dir, f'CarDataset-Splits-{i + 1}-Train.csv')
            test_file_path = os.path.join(dataset_dir, f'CarDataset-Splits-{i + 1}-Test.csv')
        # Tạo DataFrame cho tập train và test
        train_data = pd.DataFrame({
            'ImageFullPath': [path_list[idx] for idx in train_index],
            'CategoryID': [categoryid_list[idx] for idx in train_index]
        })
        test_data = pd.DataFrame({
            'ImageFullPath': [path_list[idx] for idx in test_index],
            'CategoryID': [categoryid_list[idx] for idx in test_index]
        })

        if save_csv:
            # Lưu DataFrame ra file CSV
            train_data.to_csv(train_file_path, index=False)
            test_data.to_csv(test_file_path, index=False)

            print(f"Train fold {i + 1} saved to {train_file_path}")
            print(f"Test fold {i + 1} saved to {test_file_path}")

        train_splits.append(train_data)
        test_splits.append(test_data)

    return train_splits, test_splits

In [ ]:
def display_splits(train_splits, test_splits, max_files=5):
    for i, (train_data, test_data) in enumerate(zip(train_splits, test_splits)):
        print(f"\n=================== Split {i + 1} ===================")

        print("Train Files:")
        train_files = train_data['ImageFullPath'].tolist()
        for file in train_files[:max_files]:
            print(file)
        if len(train_files) > max_files:
            print(f"... and {len(train_files) - max_files} more")

        print("\nTest Files:")
        test_files = test_data['ImageFullPath'].tolist()
        for file in test_files[:max_files]:
            print(file)
        if len(test_files) > max_files:
            print(f"... and {len(test_files) - max_files} more")


In [ ]:
def plot_class_distributions(splits, mode='train'):
    num_splits = len(splits)

    fig, axes = plt.subplots(1, num_splits, figsize=(5 * num_splits, 5), sharey=True)
    title = f"Class Distributions Across {'Training' if mode == 'train' else 'Testing'} Splits"
    fig.suptitle(title)

    for i, ax in enumerate(axes):
        class_counts = splits[i]['CategoryID'].value_counts().sort_index()
        class_counts.plot(kind='bar', ax=ax)
        ax.set_title(f'Car-Splits-{i + 1}-Train' if mode == 'train' else f'Car-Splits-{i + 1}-Test')
        ax.set_xlabel('CategoryID')
        if i == 0:
            ax.set_ylabel('Số lượng ảnh')

    plt.tight_layout()
    plt.show()

In [ ]:
def display_images(csv_file='CarDataset-Splits-1-Train.csv',
                   base_dir='./',
                   dataset_dir='./',
                   num_imgs_per_row=10,
                   img_height=150,
                   img_width=150,):
    # Đọc file CSV
    csv_path = os.path.join(dataset_dir, csv_file)
    df = pd.read_csv(csv_path)

    # Lấy danh sách các CategoryID
    categories = df['CategoryID'].unique()

    # Thiết lập vùng visualize/Điều chỉnh figsize cho phù hợp với số lượng ảnh
    fig_height = len(categories) * (img_height / 100)
    fig_width = num_imgs_per_row * (img_width / 100)
    plt.figure(figsize=(fig_height, fig_width))

    for i, category in enumerate(tqdm(categories, desc="Displaying images")):
        # Lấy ảnh thuộc category hiện tại
        category_imgs = df[df['CategoryID'] == category]['ImageFullPath'].tolist()

        # Chọn ngẫu nhiên ảnh
        selected_imgs = random.sample(category_imgs, min(len(category_imgs), num_imgs_per_row))

        # Tạo subplot cho CategoryID (đặt nó ở cột đầu tiên mỗi hàng)
        ax = plt.subplot(len(categories), num_imgs_per_row + 1, i * (num_imgs_per_row + 1) + 1)
        ax.text(0.5, 0.5, invert_indexing[category],
                ha='center', va='center', fontsize=12, fontweight='bold')
        ax.axis("off")

        # Hiển thị các ảnh trong hàng
        for j, img_path in enumerate(selected_imgs):
            ax = plt.subplot(len(categories), num_imgs_per_row + 1, i * (num_imgs_per_row + 1) + j + 2)
            try:
                img = Image.open(os.path.join(base_dir, img_path))
                img = img.resize((img_width, img_height))
                ax.imshow(img)
                ax.axis("off")
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
                ax.axis("off")

    # Điều chỉnh layout
    plt.subplots_adjust(wspace=0.2, hspace=0.4)
    plt.tight_layout()
    plt.show()


In [ ]:
display_images(csv_file='CarDataset-Splits-1-Train.csv',
               base_dir=augmented_base_dir,
               dataset_dir=dataset_dir,
               num_imgs_per_row=10,
               img_height=150,
               img_width=150)

In [ ]:
st = time.time()
augmented_extracted_features = extract_features(base_dir=augmented_base_dir,
                                      dataset_dir=dataset_dir,
                                      file_csv='AugmentedCarDataset.csv',
                                      model_name='XceptionNet',
                                      input_shape=(224, 224),
                                      partition=False,
                                      partition_size=1000,
                                      save_result=True,
                                      save_name=augmented_extracted_features_file_name,
                                      )
print(f"Total time: {(time.time()-st)/3600:.2f}h")

In [ ]:
def perform_clustering(extracted_features=None,
                       dataset_dir='./',
                       extracted_file='augmented_extracted_features.npz',
                       n_clusters=5,
                       random_state=42,
                       save_file=False,
                       save_name='augmented_clustering_Results.csv',
                       clustering_algorithm='KMeans'
                       ):
    if extracted_features is None:
        data = np.load(os.path.join(dataset_dir, extracted_file), allow_pickle=True)
        extracted_features = data['extracted_features']
    features_list = [x['Extracted Features'] for x in tqdm(extracted_features)]
    image_path_list = [x['ImageFullPath'] for x in tqdm(extracted_features)]
    category_id_list = [x['CategoryID'] for x in tqdm(extracted_features)]

    if clustering_algorithm == 'KMeans':
        clustering = KMeans(n_clusters=n_clusters, random_state=random_state).fit(features_list)
    elif clustering_algorithm == 'DBSCAN':
        clustering = DBSCAN(eps=0.5, min_samples=5).fit(features_list)
    elif clustering_algorithm == 'AgglomerativeClustering':
        clustering = AgglomerativeClustering(n_clusters=n_clusters).fit(features_list)
    elif clustering_algorithm == 'MeanShift':
        clustering = MeanShift().fit(features_list)
    elif clustering_algorithm == 'Birch':
        clustering = Birch(n_clusters=n_clusters).fit(features_list)
    elif clustering_algorithm == 'SpectralClustering':
        clustering = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors').fit(features_list)
    else:
        print(f"Clustering algorithm {clustering_algorithm} is not supported")
        return None

    df = pd.DataFrame({'ImageFullPath': image_path_list, 'ClusterID': clustering.labels_})
    if save_file:
      save_path = os.path.join(dataset_dir, save_name)
      df.to_csv(save_path, index=False)
      print(f"Clustering result saved to {save_path}")

    return df, clustering, features_list, image_path_list, category_id_list

In [ ]:
df, clustering, features_list, image_path_list, category_id_list = perform_clustering(dataset_dir=dataset_dir,
                                                                    extracted_file='/kaggle/input/cs114-augmented-features/augmented_extracted_features.npz',
                                                                    n_clusters=5,
                                                                    save_file=True,
                                                                    save_name='augmented_clustering_results.csv',
                                                                    )

 ## **Visualize kết quả**

In [ ]:
def visualize_clustering(features_list, clustering, n_components=2):
    # Lay nhan cua cluster
    unique_clusters = np.unique(clustering.labels_)

    if n_components == 2:
        # Dung PCA de giam so chieu con 2
        pca = PCA(n_components=2)
        features_2d = pca.fit_transform(features_list)

        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(features_2d[:, 0], features_2d[:, 1], c=clustering.labels_, cmap='gist_rainbow', alpha=1.0)
        plt.xlabel('PCA Component 1')
        plt.ylabel('PCA Component 2')
        plt.title(f'K-Means Clustering of Car Features on {len(features_list)} Images')

        colorbar = plt.colorbar(scatter, label='Cluster ID')
        colorbar.set_ticks(unique_clusters)
        colorbar.set_ticklabels([str(label) for label in unique_clusters])

        plt.show()

    elif n_components == 3:
        pca = PCA(n_components=3)
        features_3d = pca.fit_transform(features_list)

        df = pd.DataFrame(features_3d, columns=['PCA Component 1', 'PCA Component 2', 'PCA Component 3'])
        df['Cluster ID'] = clustering.labels_.astype(str)  # Convert to string for discrete color mapping

        fig = px.scatter_3d(df, x='PCA Component 1', y='PCA Component 2', z='PCA Component 3',
                            color='Cluster ID',
                            title=f'K-Means Clustering of Car Features on {len(features_list)} Images (3D Plot)',)

        fig.update_layout(scene=dict(xaxis_title='PCA Component 1', yaxis_title='PCA Component 2', zaxis_title='PCA Component 3'))
        fig.show()


In [ ]:
visualize_clustering(features_list, clustering, 2)

In [ ]:
def display_clustering(df=None,
                   csv_file='Clustering_results.csv',
                   base_dir='./',
                   dataset_dir='./',
                   num_imgs_per_row=10,
                   img_height=150,
                   img_width=150):
    # Đọc file CSV
    csv_path = os.path.join(dataset_dir, csv_file)
    if df is None:
        df = pd.read_csv(csv_path)

    # Lấy danh sách các ClusterID
    clusters = df['ClusterID'].unique()

    # Thiết lập vùng visualize/Điều chỉnh figsize cho phù hợp với số lượng ảnh
    fig_height = len(clusters) * (img_height / 100) * 1.2
    fig_width = num_imgs_per_row * (img_width / 100) * 1.2
    plt.figure(figsize=(fig_width, fig_height))

    for i, cluster in enumerate(tqdm(clusters, desc="Processing clusters", unit="cluster")):
        # Lấy ảnh thuộc cluster hiện tại
        cluster_imgs = df[df['ClusterID'] == cluster]['ImageFullPath'].tolist()
        # Chọn ngẫu nhiên ảnh
        selected_imgs = random.sample(cluster_imgs, min(len(cluster_imgs), num_imgs_per_row))

        # Tạo subplot cho ClusterID (đặt nó ở cột đầu tiên mỗi hàng)
        ax = plt.subplot(len(clusters), num_imgs_per_row + 1, i * (num_imgs_per_row + 1) + 1)
        ax.text(0.5, 0.5, f"Cluster {cluster}",
                ha='center', va='center', fontsize=10, fontweight='bold')
        ax.axis("off")

        # Hiển thị các ảnh trong hàng
        for j, img_path in enumerate(selected_imgs):
            ax = plt.subplot(len(clusters), num_imgs_per_row + 1, i * (num_imgs_per_row + 1) + j + 2)
            try:
                img = Image.open(os.path.join(base_dir, img_path))
                img = img.resize((img_width, img_height))
                ax.imshow(img)
                ax.axis("off")
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
                ax.axis("off")

    # Điều chỉnh layout
    plt.subplots_adjust(wspace=0.1, hspace=0.3)  # Reduce whitespace
    plt.tight_layout()
    plt.show()


In [ ]:
# display_clustering(csv_file='/kaggle/working/dataset/augmented_clustering_results.csv', base_dir=augmented_base_dir, dataset_dir=dataset_dir, num_imgs_per_row=10, img_height=150, img_width=150)

# Bước 5: Train model

In [ ]:
def load_split_with_features(csv_file,
                             features_df):
    split_df = pd.read_csv(csv_file)
    merged_df = pd.merge(split_df, features_df, on='ImageFullPath')
    merged_df = merged_df[['ImageFullPath', 'CategoryID', 'Extracted Features']]
    
    return merged_df

## **Load các đặc trưng đã được trích xuất**

In [ ]:
extracted_features = load_features('/kaggle/input/cs114-extracted-features/fulldata_extracted_features.npz')
df_extracted_features = pd.DataFrame([feature for feature in tqdm(extracted_features)])

In [ ]:
cropped_extracted_features = load_features('/kaggle/input/cs114-cropped-full-dataset/dataset/cropped_extracted_features.npz')
df_cropped_extracted_features = pd.DataFrame([feature for feature in tqdm(cropped_extracted_features)])

In [ ]:
cropped_dropdup_extracted_features = load_features('/kaggle/input/cs114-extracted-features/dropdup_extracted_features.npz')
df_cropped_dropdup_extracted_features = pd.DataFrame([feature for feature in tqdm(cropped_dropdup_extracted_features)])

In [ ]:
augmented_extracted_features = load_features('/kaggle/input/cs114-augmented-dataset/augmented_images/augmented_extracted_features.npz')
df_augmented_extracted_features = pd.DataFrame([feature for feature in tqdm(augmented_extracted_features)])

In [ ]:
full_augmented_extracted_features = load_features('/kaggle/input/cs114-full-augmented-dataset/full_augmented_extracted_features.npz')
df_full_augmented_extracted_features = pd.DataFrame([feature for feature in tqdm(full_augmented_extracted_features)])

In [ ]:
def augmented_get_dataset(base_dir='./',
                dataset_dir='/',
                categories=['Others', 'Honda', 'Hyundai', 'KIA', 'Mazda', 'Mitsubishi', 'Suzuki', 'Toyota', 'VinFast'],
                save_csv=False,
                file_name='augmented_CarDataset.csv',
                ) -> pd.DataFrame:

    os.makedirs(dataset_dir, exist_ok=True)

    path_list = []
    categoryid_list = []

    student_ids_pattern = r'(\d{8}(?:-\d{8})*)'
    categories_pattern = '|'.join(categories)
    file_extension_pattern = r'\.(jpg|jpeg|png)$'
    
    # Updated regex to handle files like "20520918.Mitsubishi.10_augmented_1.jpg"
    accepted_filename = re.compile(fr'{student_ids_pattern}\.({categories_pattern})\.[\w-]+{file_extension_pattern}')

    for category in tqdm(categories, desc="Processing categories"):
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            for filename in os.listdir(category_path):
                match = accepted_filename.match(filename)
                if match:
                    _, car_category, _ = match.groups()
                    if car_category in categories:
                        full_path = os.path.join(category, filename)  # Relative path within base_dir
                        path_list.append(full_path)
                        categoryid_list.append(indexing[car_category])

    df = pd.DataFrame({
        'ImageFullPath': path_list,
        'CategoryID': categoryid_list
    })

    if save_csv:
        output_file = os.path.join(dataset_dir, file_name)
        df.to_csv(output_file, index=False)
        print(f"{file_name} saved to {output_file}")

    return df

## **Tạo các Dataframe để tiến hành Split dữ liệu**

In [ ]:
data = get_dataset(base_dir=cropped_base_dir,
                            dataset_dir=dataset_dir,
                            file_name=dataset_name,
                            save_csv=True)
cropped_data = get_dataset(base_dir=cropped_base_dir,
                            dataset_dir=dataset_dir,
                            file_name=cropped_dataset_name,
                            save_csv=True)
cropped_dropdup_data = get_dataset(base_dir=cropped_base_dir,
                            dataset_dir=dataset_dir,
                            file_name=cropped_dropdup_dataset_name,
                            save_csv=True)
augmented_data = augmented_get_dataset(base_dir=augmented_base_dir,
                            dataset_dir=dataset_dir,
                            file_name=augmented_dataset_name,
                            save_csv=True)
full_augmented_data = augmented_get_dataset(base_dir=full_augmented_base_dir,
                            dataset_dir=dataset_dir,
                            file_name=full_augmented_dataset_name,
                            save_csv=True)

In [ ]:
train_splits, test_splits = split_kfold_datasets(num_splits=num_splits,
                                                 dataset_dir=dataset_dir,
                                                 file_name=dataset_name,
                                                 save_csv=True,
                                                 random_state=42)
cropped_train_splits, cropped_test_splits = split_kfold_datasets(num_splits=num_splits,
                                                 dataset_dir=dataset_dir,
                                                 file_name=cropped_dataset_name,
                                                 save_csv=True,
                                                 prefix='cropped',
                                                 random_state=42)
cropped_dropdup_train_splits, cropped_dropdup_test_splits = split_kfold_datasets(num_splits=num_splits,
                                                 dataset_dir=dataset_dir,
                                                 file_name=cropped_dropdup_dataset_name,
                                                 save_csv=True,
                                                 prefix='cropped_dropdup',                                                                              
                                                 random_state=42)
augmented_train_splits, augmented_test_splits = split_kfold_datasets(num_splits=num_splits,
                                                 dataset_dir=dataset_dir,
                                                 file_name=augmented_dataset_name,
                                                 prefix='augmented',
                                                 save_csv=True,
                                                 random_state=42)
full_augmented_train_splits, full_augmented_test_splits = split_kfold_datasets(num_splits=num_splits,
                                                 dataset_dir=dataset_dir,
                                                 file_name=full_augmented_dataset_name,
                                                 prefix='full_augmented',
                                                 save_csv=True,
                                                 random_state=42)

In [ ]:
display_splits(train_splits, test_splits)

In [ ]:
plot_class_distributions(train_splits, mode='train')
plot_class_distributions(test_splits, mode='test')

In [ ]:
display_splits(cropped_train_splits, cropped_test_splits)

In [ ]:
plot_class_distributions(cropped_train_splits, mode='train')
plot_class_distributions(cropped_test_splits, mode='test')

In [ ]:
display_splits(cropped_dropdup_train_splits, cropped_dropdup_test_splits)

In [ ]:
plot_class_distributions(cropped_dropdup_train_splits, mode='train')
plot_class_distributions(cropped_dropdup_test_splits, mode='test')

In [ ]:
display_splits(augmented_train_splits, augmented_test_splits)

In [ ]:
plot_class_distributions(augmented_train_splits, mode='train')
plot_class_distributions(augmented_test_splits, mode='test')

In [ ]:
display_splits(full_augmented_train_splits, full_augmented_test_splits)

In [ ]:
plot_class_distributions(full_augmented_train_splits, mode='train')
plot_class_distributions(full_augmented_test_splits, mode='test')

## **Đưa dữ liệu từ các splits vào các biến**

In [ ]:
def load_and_prepare_data(train_files, test_files, features_df):
    train_splits = []
    test_splits = []

    def load_split_with_features(csv_file, features_df):
        split_df = pd.read_csv(csv_file)
        merged_df = pd.merge(split_df, features_df, on='ImageFullPath')
        merged_df = merged_df[['ImageFullPath', 'CategoryID', 'Extracted Features']]
        return merged_df

    for train_file in train_files:
      train_split_df = load_split_with_features(train_file, features_df)
      train_splits.append(train_split_df)

    for test_file in test_files:
      test_split_df = load_split_with_features(test_file, features_df)
      test_splits.append(test_split_df)

    return train_splits, test_splits

In [ ]:
df_features = df_extracted_features.drop('CategoryID', axis=1)
df_cropped_features = df_cropped_extracted_features.drop('CategoryID', axis=1)
df_cropped_dropdup_features = df_cropped_dropdup_extracted_features.drop('CategoryID', axis=1)
df_augmented_features = df_augmented_extracted_features.drop('CategoryID', axis=1)
df_full_augmented_features = df_full_augmented_extracted_features.drop('CategoryID', axis=1)

In [ ]:
# Original Dataset
train_files = [f'{dataset_dir}/CarDataset-Splits-{i + 1}-Train.csv' for i in tqdm(range(num_splits), desc="Generating Original Train File")]
test_files = [f'{dataset_dir}/CarDataset-Splits-{i + 1}-Test.csv' for i in tqdm(range(num_splits), desc="Generating Original Test File")]
train_splits_with_features, test_splits_with_features = load_and_prepare_data(train_files, test_files, df_features)

# Cropped Dataset
cropped_train_files = [f'{dataset_dir}/cropped_CarDataset-Splits-{i + 1}-Train.csv' for i in tqdm(range(num_splits), desc="Generating Cropped Train File")]
cropped_test_files = [f'{dataset_dir}/cropped_CarDataset-Splits-{i + 1}-Test.csv' for i in tqdm(range(num_splits), desc="Generating Cropped Test File")]
cropped_train_splits_with_features, cropped_test_splits_with_features = load_and_prepare_data(cropped_train_files, cropped_test_files, df_cropped_features)

# Cropped and Deduplicated Dataset
cropped_dropdup_train_files = [f'{dataset_dir}/cropped_dropdup_CarDataset-Splits-{i + 1}-Train.csv' for i in tqdm(range(num_splits), desc="Generating Cropped Dropdup Train File")]
cropped_dropdup_test_files = [f'{dataset_dir}/cropped_dropdup_CarDataset-Splits-{i + 1}-Test.csv' for i in tqdm(range(num_splits), desc="Generating Cropped Dropdup Test File")]
cropped_dropdup_train_splits_with_features, cropped_dropdup_test_splits_with_features = load_and_prepare_data(cropped_dropdup_train_files, cropped_dropdup_test_files, df_cropped_dropdup_features)

# Augmented Dataset
augmented_train_files = [f'{dataset_dir}/augmented_CarDataset-Splits-{i + 1}-Train.csv' for i in tqdm(range(num_splits), desc="Generating Augmented Train File")]
augmented_test_files = [f'{dataset_dir}/augmented_CarDataset-Splits-{i + 1}-Test.csv' for i in tqdm(range(num_splits), desc="Generating Augmented Test File")]
augmented_train_splits_with_features, augmented_test_splits_with_features = load_and_prepare_data(augmented_train_files, augmented_test_files, df_augmented_features)

# Full Augmented Dataset
full_augmented_train_files = [f'{dataset_dir}/full_augmented_CarDataset-Splits-{i + 1}-Train.csv' for i in tqdm(range(num_splits), desc="Generating Full Augmented Train File")]
full_augmented_test_files = [f'{dataset_dir}/full_augmented_CarDataset-Splits-{i + 1}-Test.csv' for i in tqdm(range(num_splits), desc="Generating Full Augmented Test File")]
full_augmented_train_splits_with_features, full_augmented_test_splits_with_features = load_and_prepare_data(full_augmented_train_files, augmented_test_files, df_augmented_features)

In [ ]:
# Full Dataset
train_df_1 = train_splits_with_features[0]
train_df_2 = train_splits_with_features[1]
train_df_3 = train_splits_with_features[2]
train_df_4 = train_splits_with_features[3]
train_df_5 = train_splits_with_features[4]

test_df_1 = test_splits_with_features[0]
test_df_2 = test_splits_with_features[1]
test_df_3 = test_splits_with_features[2]
test_df_4 = test_splits_with_features[3]
test_df_5 = test_splits_with_features[4]

# Cropped Dataset
cropped_train_df_1 = cropped_train_splits_with_features[0]
cropped_train_df_2 = cropped_train_splits_with_features[1]
cropped_train_df_3 = cropped_train_splits_with_features[2]
cropped_train_df_4 = cropped_train_splits_with_features[3]
cropped_train_df_5 = cropped_train_splits_with_features[4]

cropped_test_df_1 = cropped_test_splits_with_features[0]
cropped_test_df_2 = cropped_test_splits_with_features[1]
cropped_test_df_3 = cropped_test_splits_with_features[2]
cropped_test_df_4 = cropped_test_splits_with_features[3]
cropped_test_df_5 = cropped_test_splits_with_features[4]

# Cropped + Drop Duplicate Dataset
cropped_dropdup_train_df_1 = cropped_dropdup_train_splits_with_features[0]
cropped_dropdup_train_df_2 = cropped_dropdup_train_splits_with_features[1]
cropped_dropdup_train_df_3 = cropped_dropdup_train_splits_with_features[2]
cropped_dropdup_train_df_4 = cropped_dropdup_train_splits_with_features[3]
cropped_dropdup_train_df_5 = cropped_dropdup_train_splits_with_features[4]

cropped_dropdup_test_df_1 = cropped_dropdup_test_splits_with_features[0]
cropped_dropdup_test_df_2 = cropped_dropdup_test_splits_with_features[1]
cropped_dropdup_test_df_3 = cropped_dropdup_test_splits_with_features[2]
cropped_dropdup_test_df_4 = cropped_dropdup_test_splits_with_features[3]
cropped_dropdup_test_df_5 = cropped_dropdup_test_splits_with_features[4]

# Augmented Dataset
augmented_train_df_1 = augmented_train_splits_with_features[0]
augmented_train_df_2 = augmented_train_splits_with_features[1]
augmented_train_df_3 = augmented_train_splits_with_features[2]
augmented_train_df_4 = augmented_train_splits_with_features[3]
augmented_train_df_5 = augmented_train_splits_with_features[4]

augmented_test_df_1 = augmented_test_splits_with_features[0]
augmented_test_df_2 = augmented_test_splits_with_features[1]
augmented_test_df_3 = augmented_test_splits_with_features[2]
augmented_test_df_4 = augmented_test_splits_with_features[3]
augmented_test_df_5 = augmented_test_splits_with_features[4]

# Full Augmented Dataset
full_augmented_train_df_1 = full_augmented_train_splits_with_features[0]
full_augmented_train_df_2 = full_augmented_train_splits_with_features[1]
full_augmented_train_df_3 = full_augmented_train_splits_with_features[2]
full_augmented_train_df_4 = full_augmented_train_splits_with_features[3]
full_augmented_train_df_5 = full_augmented_train_splits_with_features[4]

full_augmented_test_df_1 = full_augmented_test_splits_with_features[0]
full_augmented_test_df_2 = full_augmented_test_splits_with_features[1]
full_augmented_test_df_3 = full_augmented_test_splits_with_features[2]
full_augmented_test_df_4 = full_augmented_test_splits_with_features[3]
full_augmented_test_df_5 = full_augmented_test_splits_with_features[4]

In [ ]:

X_train_1 = np.array(train_df_1['Extracted Features'].apply(lambda x: x).tolist())
X_train_2 = np.array(train_df_2['Extracted Features'].apply(lambda x: x).tolist())
X_train_3 = np.array(train_df_3['Extracted Features'].apply(lambda x: x).tolist())
X_train_4 = np.array(train_df_4['Extracted Features'].apply(lambda x: x).tolist())
X_train_5 = np.array(train_df_5['Extracted Features'].apply(lambda x: x).tolist())

X_test_1 = np.array(test_df_1['Extracted Features'].apply(lambda x: x).tolist())
X_test_2 = np.array(test_df_2['Extracted Features'].apply(lambda x: x).tolist())
X_test_3 = np.array(test_df_3['Extracted Features'].apply(lambda x: x).tolist())
X_test_4 = np.array(test_df_4['Extracted Features'].apply(lambda x: x).tolist())
X_test_5 = np.array(test_df_5['Extracted Features'].apply(lambda x: x).tolist())

y_train_1 = np.array(train_df_1['CategoryID'].tolist())
y_train_2 = np.array(train_df_2['CategoryID'].tolist())
y_train_3 = np.array(train_df_3['CategoryID'].tolist())
y_train_4 = np.array(train_df_4['CategoryID'].tolist())
y_train_5 = np.array(train_df_5['CategoryID'].tolist())

y_test_1 = np.array(test_df_1['CategoryID'].tolist())
y_test_2 = np.array(test_df_2['CategoryID'].tolist())
y_test_3 = np.array(test_df_3['CategoryID'].tolist())
y_test_4 = np.array(test_df_4['CategoryID'].tolist())
y_test_5 = np.array(test_df_5['CategoryID'].tolist())


In [ ]:

cropped_X_train_1 = np.array(cropped_train_df_1['Extracted Features'].apply(lambda x: x).tolist())
cropped_X_train_2 = np.array(cropped_train_df_2['Extracted Features'].apply(lambda x: x).tolist())
cropped_X_train_3 = np.array(cropped_train_df_3['Extracted Features'].apply(lambda x: x).tolist())
cropped_X_train_4 = np.array(cropped_train_df_4['Extracted Features'].apply(lambda x: x).tolist())
cropped_X_train_5 = np.array(cropped_train_df_5['Extracted Features'].apply(lambda x: x).tolist())

cropped_X_test_1 = np.array(cropped_test_df_1['Extracted Features'].apply(lambda x: x).tolist())
cropped_X_test_2 = np.array(cropped_test_df_2['Extracted Features'].apply(lambda x: x).tolist())
cropped_X_test_3 = np.array(cropped_test_df_3['Extracted Features'].apply(lambda x: x).tolist())
cropped_X_test_4 = np.array(cropped_test_df_4['Extracted Features'].apply(lambda x: x).tolist())
cropped_X_test_5 = np.array(cropped_test_df_5['Extracted Features'].apply(lambda x: x).tolist())

cropped_y_train_1 = np.array(cropped_train_df_1['CategoryID'].tolist())
cropped_y_train_2 = np.array(cropped_train_df_2['CategoryID'].tolist())
cropped_y_train_3 = np.array(cropped_train_df_3['CategoryID'].tolist())
cropped_y_train_4 = np.array(cropped_train_df_4['CategoryID'].tolist())
cropped_y_train_5 = np.array(cropped_train_df_5['CategoryID'].tolist())

cropped_y_test_1 = np.array(cropped_test_df_1['CategoryID'].tolist())
cropped_y_test_2 = np.array(cropped_test_df_2['CategoryID'].tolist())
cropped_y_test_3 = np.array(cropped_test_df_3['CategoryID'].tolist())
cropped_y_test_4 = np.array(cropped_test_df_4['CategoryID'].tolist())
cropped_y_test_5 = np.array(cropped_test_df_5['CategoryID'].tolist())


In [ ]:
cropped_dropdup_X_train_1 = np.array(cropped_dropdup_train_df_1['Extracted Features'].apply(lambda x: x).tolist())
cropped_dropdup_X_train_2 = np.array(cropped_dropdup_train_df_2['Extracted Features'].apply(lambda x: x).tolist())
cropped_dropdup_X_train_3 = np.array(cropped_dropdup_train_df_3['Extracted Features'].apply(lambda x: x).tolist())
cropped_dropdup_X_train_4 = np.array(cropped_dropdup_train_df_4['Extracted Features'].apply(lambda x: x).tolist())
cropped_dropdup_X_train_5 = np.array(cropped_dropdup_train_df_5['Extracted Features'].apply(lambda x: x).tolist())

cropped_dropdup_X_test_1 = np.array(cropped_dropdup_test_df_1['Extracted Features'].apply(lambda x: x).tolist())
cropped_dropdup_X_test_2 = np.array(cropped_dropdup_test_df_2['Extracted Features'].apply(lambda x: x).tolist())
cropped_dropdup_X_test_3 = np.array(cropped_dropdup_test_df_3['Extracted Features'].apply(lambda x: x).tolist())
cropped_dropdup_X_test_4 = np.array(cropped_dropdup_test_df_4['Extracted Features'].apply(lambda x: x).tolist())
cropped_dropdup_X_test_5 = np.array(cropped_dropdup_test_df_5['Extracted Features'].apply(lambda x: x).tolist())

cropped_dropdup_y_train_1 = np.array(cropped_dropdup_train_df_1['CategoryID'].tolist())
cropped_dropdup_y_train_2 = np.array(cropped_dropdup_train_df_2['CategoryID'].tolist())
cropped_dropdup_y_train_3 = np.array(cropped_dropdup_train_df_3['CategoryID'].tolist())
cropped_dropdup_y_train_4 = np.array(cropped_dropdup_train_df_4['CategoryID'].tolist())
cropped_dropdup_y_train_5 = np.array(cropped_dropdup_train_df_5['CategoryID'].tolist())

cropped_dropdup_y_test_1 = np.array(cropped_dropdup_test_df_1['CategoryID'].tolist())
cropped_dropdup_y_test_2 = np.array(cropped_dropdup_test_df_2['CategoryID'].tolist())
cropped_dropdup_y_test_3 = np.array(cropped_dropdup_test_df_3['CategoryID'].tolist())
cropped_dropdup_y_test_4 = np.array(cropped_dropdup_test_df_4['CategoryID'].tolist())
cropped_dropdup_y_test_5 = np.array(cropped_dropdup_test_df_5['CategoryID'].tolist())


In [ ]:
augmented_X_train_1 = np.array(augmented_train_df_1['Extracted Features'].apply(lambda x: x).tolist())
augmented_X_train_2 = np.array(augmented_train_df_2['Extracted Features'].apply(lambda x: x).tolist())
augmented_X_train_3 = np.array(augmented_train_df_3['Extracted Features'].apply(lambda x: x).tolist())
augmented_X_train_4 = np.array(augmented_train_df_4['Extracted Features'].apply(lambda x: x).tolist())
augmented_X_train_5 = np.array(augmented_train_df_5['Extracted Features'].apply(lambda x: x).tolist())

augmented_X_test_1 = np.array(augmented_test_df_1['Extracted Features'].apply(lambda x: x).tolist())
augmented_X_test_2 = np.array(augmented_test_df_2['Extracted Features'].apply(lambda x: x).tolist())
augmented_X_test_3 = np.array(augmented_test_df_3['Extracted Features'].apply(lambda x: x).tolist())
augmented_X_test_4 = np.array(augmented_test_df_4['Extracted Features'].apply(lambda x: x).tolist())
augmented_X_test_5 = np.array(augmented_test_df_5['Extracted Features'].apply(lambda x: x).tolist())

augmented_y_train_1 = np.array(augmented_train_df_1['CategoryID'].tolist())
augmented_y_train_2 = np.array(augmented_train_df_2['CategoryID'].tolist())
augmented_y_train_3 = np.array(augmented_train_df_3['CategoryID'].tolist())
augmented_y_train_4 = np.array(augmented_train_df_4['CategoryID'].tolist())
augmented_y_train_5 = np.array(augmented_train_df_5['CategoryID'].tolist())

augmented_y_test_1 = np.array(augmented_test_df_1['CategoryID'].tolist())
augmented_y_test_2 = np.array(augmented_test_df_2['CategoryID'].tolist())
augmented_y_test_3 = np.array(augmented_test_df_3['CategoryID'].tolist())
augmented_y_test_4 = np.array(augmented_test_df_4['CategoryID'].tolist())
augmented_y_test_5 = np.array(augmented_test_df_5['CategoryID'].tolist())

In [ ]:
full_augmented_X_train_1 = np.array(full_augmented_train_df_1['Extracted Features'].apply(lambda x: x).tolist())
full_augmented_X_train_2 = np.array(full_augmented_train_df_2['Extracted Features'].apply(lambda x: x).tolist())
full_augmented_X_train_3 = np.array(full_augmented_train_df_3['Extracted Features'].apply(lambda x: x).tolist())
full_augmented_X_train_4 = np.array(full_augmented_train_df_4['Extracted Features'].apply(lambda x: x).tolist())
full_augmented_X_train_5 = np.array(full_augmented_train_df_5['Extracted Features'].apply(lambda x: x).tolist())

full_augmented_X_test_1 = np.array(full_augmented_test_df_1['Extracted Features'].apply(lambda x: x).tolist())
full_augmented_X_test_2 = np.array(full_augmented_test_df_2['Extracted Features'].apply(lambda x: x).tolist())
full_augmented_X_test_3 = np.array(full_augmented_test_df_3['Extracted Features'].apply(lambda x: x).tolist())
full_augmented_X_test_4 = np.array(full_augmented_test_df_4['Extracted Features'].apply(lambda x: x).tolist())
full_augmented_X_test_5 = np.array(full_augmented_test_df_5['Extracted Features'].apply(lambda x: x).tolist())

full_augmented_y_train_1 = np.array(full_augmented_train_df_1['CategoryID'].tolist())
full_augmented_y_train_2 = np.array(full_augmented_train_df_2['CategoryID'].tolist())
full_augmented_y_train_3 = np.array(full_augmented_train_df_3['CategoryID'].tolist())
full_augmented_y_train_4 = np.array(full_augmented_train_df_4['CategoryID'].tolist())
full_augmented_y_train_5 = np.array(full_augmented_train_df_5['CategoryID'].tolist())

full_augmented_y_test_1 = np.array(full_augmented_test_df_1['CategoryID'].tolist())
full_augmented_y_test_2 = np.array(full_augmented_test_df_2['CategoryID'].tolist())
full_augmented_y_test_3 = np.array(full_augmented_test_df_3['CategoryID'].tolist())
full_augmented_y_test_4 = np.array(full_augmented_test_df_4['CategoryID'].tolist())
full_augmented_y_test_5 = np.array(full_augmented_test_df_5['CategoryID'].tolist())


## **Tiến hành huấn luyện và đánh giá**

### GPU
Chỉ hoạt động với T4 GPU

In [ ]:
datasets = [
    (X_train_1, y_train_1, X_test_1, y_test_1),
    (X_train_2, y_train_2, X_test_2, y_test_2),
    (X_train_3, y_train_3, X_test_3, y_test_3),
    (X_train_4, y_train_4, X_test_4, y_test_4),
    (X_train_5, y_train_5, X_test_5, y_test_5)
]
cropped_datasets = [
    (cropped_X_train_1, cropped_y_train_1, cropped_X_test_1, cropped_y_test_1),
    (cropped_X_train_2, cropped_y_train_2, cropped_X_test_2, cropped_y_test_2),
    (cropped_X_train_3, cropped_y_train_3, cropped_X_test_3, cropped_y_test_3),
    (cropped_X_train_4, cropped_y_train_4, cropped_X_test_4, cropped_y_test_4),
    (cropped_X_train_5, cropped_y_train_5, cropped_X_test_5, cropped_y_test_5)
]

cropped_dropdup_datasets = [
    (cropped_dropdup_X_train_1, cropped_dropdup_y_train_1, cropped_dropdup_X_test_1, cropped_dropdup_y_test_1),
    (cropped_dropdup_X_train_2, cropped_dropdup_y_train_2, cropped_dropdup_X_test_2, cropped_dropdup_y_test_2),
    (cropped_dropdup_X_train_3, cropped_dropdup_y_train_3, cropped_dropdup_X_test_3, cropped_dropdup_y_test_3),
    (cropped_dropdup_X_train_4, cropped_dropdup_y_train_4, cropped_dropdup_X_test_4, cropped_dropdup_y_test_4),
    (cropped_dropdup_X_train_5, cropped_dropdup_y_train_5, cropped_dropdup_X_test_5, cropped_dropdup_y_test_5)
]

augmented_datasets = [
    (augmented_X_train_1, augmented_y_train_1, augmented_X_test_1, augmented_y_test_1),
    (augmented_X_train_2, augmented_y_train_2, augmented_X_test_2, augmented_y_test_2),
    (augmented_X_train_3, augmented_y_train_3, augmented_X_test_3, augmented_y_test_3),
    (augmented_X_train_4, augmented_y_train_4, augmented_X_test_4, augmented_y_test_4),
    (augmented_X_train_5, augmented_y_train_5, augmented_X_test_5, augmented_y_test_5)
]

full_augmented_datasets = [
    (full_augmented_X_train_1, full_augmented_y_train_1, full_augmented_X_test_1, full_augmented_y_test_1),
    (full_augmented_X_train_2, full_augmented_y_train_2, full_augmented_X_test_2, full_augmented_y_test_2),
    (full_augmented_X_train_3, full_augmented_y_train_3, full_augmented_X_test_3, full_augmented_y_test_3),
    (full_augmented_X_train_4, full_augmented_y_train_4, full_augmented_X_test_4, full_augmented_y_test_4),
    (full_augmented_X_train_5, full_augmented_y_train_5, full_augmented_X_test_5, full_augmented_y_test_5)
]

def fitting_datasets(datasets, prefix, model_name):
    results = []
    if model_name == 'SVM':
        model = cuSVC(kernel='rbf', random_state=42)
    elif model_name == 'RF':
        model = cuRFClassifier(random_state=42)
    else:
        model = cuKNNClassifier(random_state=42)
    for i, (X_train, y_train, X_test, y_test) in enumerate(tqdm(datasets, desc=f"Training {prefix} models")):
        with tqdm(total=2, desc=f"Fitting {model_name} on {prefix} Dataset {i+1}", leave=False) as pbar:
            model.fit(X_train, y_train)
            pbar.update(1)
            
            # Make predictions
            y_pred = model.predict(X_test)
            pbar.update(1)
        
        accuracy = accuracy_score(y_test, y_pred)
        results.append(accuracy)
        print(f'{model_name} Accuracy for {prefix} dataset {i+1}: {accuracy:.6f}')
    
    average_accuracy = sum(results) / len(results)
    print(f'Average {model_name} Accuracy for {prefix} datasets: {average_accuracy:.6f}')
    
    return model, results, average_accuracy

In [ ]:
svm_original_model, svm_original_results, svm_average_accuracy = fitting_datasets(datasets, 'original', 'SVM')
svm_cropped_model, svm_cropped_results, svm_cropped_average_accuracy = fitting_datasets(cropped_datasets, 'cropped', 'SVM')
svm_cropped_dropdup_model, svm_cropped_dropdup_results, svm_cropped_dropdup_average_accuracy = fitting_datasets(cropped_dropdup_datasets, 'cropped_dropdup', 'SVM')
svm_augmented_model, svm_augmented_results, svm_augmented_average_accuracy = fitting_datasets(augmented_datasets, 'augmented', 'SVM')
svm_full_augmented_model, svm_full_augmented_results, svm_full_augmented_average_accuracy = fitting_datasets(full_augmented_datasets, 'full_augmented', 'SVM')

In [ ]:
rf_original_model, rf_original_results, rf_average_accuracy = fitting_datasets(datasets, 'original', 'RF')
rf_cropped_model, rf_cropped_results, rf_cropped_average_accuracy = fitting_datasets(cropped_datasets, 'cropped', 'RF')
rf_cropped_dropdup_model, rf_cropped_dropdup_results, rf_cropped_dropdup_average_accuracy = fitting_datasets(cropped_dropdup_datasets, 'cropped_dropdup', 'RF')
rf_augmented_model, rf_augmented_results, rf_augmented_average_accuracy = fitting_datasets(augmented_datasets, 'augmented', 'RF')
rf_full_augmented_model, rf_full_augmented_results, rf_full_augmented_average_accuracy = fitting_datasets(full_augmented_datasets, 'full_augmented', 'RF')

In [ ]:
knn_original_model, knn_original_results, knn_average_accuracy = fitting_datasets(datasets, 'original', 'KNN')
knn_cropped_model, knn_cropped_results, knn_cropped_average_accuracy = fitting_datasets(cropped_datasets, 'cropped', 'KNN')
knn_cropped_dropdup_model, knn_cropped_dropdup_results, knn_cropped_dropdup_average_accuracy = fitting_datasets(cropped_dropdup_datasets, 'cropped_dropdup', 'KNN')
knn_augmented_model, knn_augmented_results, knn_augmented_average_accuracy = fitting_datasets(augmented_datasets, 'augmented', 'KNN')
knn_full_augmented_model, knn_full_augmented_results, knn_full_augmented_average_accuracy = fitting_datasets(full_augmented_datasets, 'full_augmented', 'KNN')

## **Visualize Kết quả**

In [ ]:
def plot_confusion_matrix(y_true,
                          y_pred,
                          labels,
                          title,
                          prefix=None):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    title = prefix + '_' + title if prefix is not None else prefix
    plt.title(title)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()
    

In [ ]:
_, _, X_test, y_test = datasets[-1]
svm_y_pred = svm_original_model.predict(X_test)
rf_y_pred = rf_original_model.predict(X_test)
knn_y_pred = knn_original_model.predict(X_test)

plot_confusion_matrix(y_test,
                      svm_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for SVM (Dataset 5)',
                      prefix='full')

plot_confusion_matrix(y_test,
                      rf_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for Random Forest (Dataset 5)',
                      prefix='full')

plot_confusion_matrix(y_test,
                      knn_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for KNN (Dataset 5)',
                      prefix='full')


In [ ]:
_, _, cropped_X_test, cropped_y_test = cropped_datasets[-1]
svm_y_pred = svm_cropped_model.predict(cropped_X_test)
rf_y_pred = rf_cropped_model.predict(cropped_X_test)
knn_y_pred = knn_cropped_model.predict(cropped_X_test)

plot_confusion_matrix(cropped_y_test,
                      svm_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for SVM (Dataset 5)',
                      prefix='cropped')

plot_confusion_matrix(cropped_y_test,
                      rf_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for Random Forest (Dataset 5)',
                      prefix='cropped')

plot_confusion_matrix(cropped_y_test,
                      knn_y_pred,
                      labels=[0, 1],
                      title='Confusion Matrix for KNN (Dataset 5)',
                      prefix='cropped')

In [ ]:
_, _, cropped_dropdup_X_test, cropped_dropdup_y_test = cropped_dropdup_datasets[-1]
svm_y_pred = svm_cropped_dropdup_model.predict(cropped_dropdup_X_test)
rf_y_pred = rf_cropped_dropdup_model.predict(cropped_dropdup_X_test)
knn_y_pred = knn_cropped_dropdup_model.predict(cropped_dropdup_X_test)

plot_confusion_matrix(cropped_dropdup_y_test,
                      svm_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for SVM (Dataset 5)',
                      prefix='cropped_dropdup')

plot_confusion_matrix(cropped_dropdup_y_test,
                      rf_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for Random Forest (Dataset 5)',
                      prefix='cropped_dropdup')

plot_confusion_matrix(cropped_dropdup_y_test,
                      knn_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for KNN (Dataset 5)',
                      prefix='cropped_dropdup')


In [ ]:
_, _, augmented_X_test, augmented_y_test = augmented_datasets[-1]
svm_y_pred = svm_augmented_model.predict(augmented_X_test)
rf_y_pred = rf_augmented_model.predict(augmented_X_test)
knn_y_pred = knn_augmented_model.predict(augmented_X_test)

plot_confusion_matrix(augmented_y_test,
                      svm_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for SVM (Dataset 5)',
                      prefix='augmented')

plot_confusion_matrix(augmented_y_test,
                      rf_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for Random Forest (Dataset 5)',
                      prefix='augmented')

plot_confusion_matrix(augmented_y_test,
                      knn_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for KNN (Dataset 5)',
                      prefix='augmented')

In [ ]:
_, _, full_augmented_X_test, full_augmented_y_test = full_augmented_datasets[-1]
svm_y_pred = svm_full_augmented_model.predict(full_augmented_X_test)
rf_y_pred = rf_full_augmented_model.predict(full_augmented_X_test)
knn_y_pred = knn_full_augmented_model.predict(full_augmented_X_test)

plot_confusion_matrix(full_augmented_y_test,
                      svm_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for SVM (Dataset 5)',
                      prefix='full_augmented')

plot_confusion_matrix(full_augmented_y_test,
                      rf_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for Random Forest (Dataset 5)',
                      prefix='full_augmented')

plot_confusion_matrix(full_augmented_y_test,
                      knn_y_pred,
                      labels=[label for label in range(len(categories))],
                      title='Confusion Matrix for KNN (Dataset 5)',
                      prefix='full_augmented')

In [ ]:
def visualize_predictions(base_dir,
                          image_paths,
                          y_true,
                          model,
                          X_test,
                          categories,
                          num_images=5):
    y_pred = model.predict(X_test)

    correct_indices = np.where(y_pred == y_true)[0]
    incorrect_indices = np.where(y_pred != y_true)[0]

    def display_images(indices, title):
        plt.figure(figsize=(12, 5))
        plt.suptitle(title, fontsize=16)
        for i, idx in enumerate(random.sample(list(indices), min(num_images, len(indices)))):
            plt.subplot(1, num_images, i + 1)
            img_path = os.path.join(base_dir, image_paths[idx])
            try:
                img = Image.open(img_path)
                plt.imshow(img)
                plt.title(f"True Label: {categories[y_true[idx]]}\nPredicted Label: {categories[y_pred[idx]]}")
                plt.axis("off")
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
        plt.tight_layout()
        plt.show()

    print("Visualizing Correctly Classified Images:")
    display_images(correct_indices, "Correctly Classified Images")

    print("Visualizing Incorrectly Classified Images:")
    display_images(incorrect_indices, "Incorrectly Classified Images")

In [ ]:
visualize_predictions(base_dir=base_dir,
                      image_paths=test_df_1['ImageFullPath'].tolist(),
                      y_true=y_test_1,
                      model=svm_original_model,
                      X_test=X_test_1,
                      categories=categories,
                      num_images=5)

In [ ]:
visualize_predictions(base_dir=cropped_base_dir,
                      image_paths=cropped_test_df_1['ImageFullPath'].tolist(),
                      y_true=cropped_y_test_1,
                      model=svm_cropped_model,
                      X_test=cropped_X_test_1,
                      categories=categories,
                      num_images=5)

In [ ]:
visualize_predictions(base_dir=cropped_dropdup_base_dir,
                      image_paths=cropped_dropdup_test_df_1['ImageFullPath'].tolist(),
                      y_true=cropped_dropdup_y_test_1,
                      model=svm_cropped_dropdup_model,
                      X_test=cropped_dropdup_X_test_1,
                      categories=categories,
                      num_images=5)

In [ ]:
visualize_predictions(base_dir=augmented_base_dir,
                      image_paths=augmented_test_df_1['ImageFullPath'].tolist(),
                      y_true=augmented_y_test_1,
                      model=svm_augmented_model,
                      X_test=augmented_X_test_1,
                      categories=categories,
                      num_images=5)

In [ ]:
visualize_predictions(base_dir=full_augmented_base_dir,
                      image_paths=full_augmented_test_df_1['ImageFullPath'].tolist(),
                      y_true=full_augmented_y_test_1,
                      model=svm_full_augmented_model,
                      X_test=full_augmented_X_test_1,
                      categories=categories,
                      num_images=5)